# EE603 Coding Assignment
- Use python3
- Submit your "rendered" ipynb, i.e., with outputs of codes (plots and printed values) visible below
- Do not change the return variables, as the evaluation is done by test cases based on the variables specified. Only add your code at "### WRITE YOUR CODE HERE"
- Use only numpy and librosa library for computing and signal processing, no other package allowed
- If you are using your mobile phone, you can use colab.research.google.com for coding
- Do not define multiple functions using same name. We will be using eval.py to auto evaluate your codes. Please check with sample test cases before submitting. We will share the evaluation test cases with you after the submission deadline.
- While submitting this file, change file name from 'YourRollNo.ipynb' to your actual roll no (Eg. 18204279.ipynb)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import librosa as lb
import pickle

In [5]:
import glob as glob
def readDir(dirname, Fs = 16000):
    
    '''
    Each audio clip should be upto 10s long; split larger audio files into many clips (non-overlapping) 

    Use load_audio(file) 
    
    Inputs: 
        dirname: (str) directory name
        Fs: (int) sampling rate
    Output: 
        x: np arrays of shape (Nclips, Nsamples) Nsamples correspond to 10s length. Use zero-padding for shorter clips.
    '''  


    ### WRITE YOUR CODE HERE - 5 MARKS
    x=[]
    samples_10s=Fs*10
    files=glob.glob(dirname+'\*.wav')
    for i in range(len(files)):
        a=load_audio(files[i])
        No_sample=a.size
        
        if No_sample%samples_10s==0:
            a=a
        else:
            t=int(No_sample/samples_10s)
            no_zerostopad=samples_10s*(t+1)-No_sample
            a=np.append(a,[np.zeros(no_zerostopad)])
        
        frames=len(a)/samples_10s
        a=np.array_split(a,frames)
        x.extend(a)
    x=np.array(x)
        
    return x


In [6]:
def load_audio(filename, Fs = 16000):
    '''
    Inputs: 
        filename: (str) filename
        Fs: (int) sampling rate
    Output: 
        x: 1D np array 
    '''
    

    ### WRITE YOUR CODE HERE - 2 MARKS
    x,sr=lb.load(filename,sr=Fs)
    
    return x

In [7]:
def splitData(X, t, testFraction=0.2, randomize = False):
    """
    Split the data randomly into training and test sets
    Use numpy functions only
    Inputs:
        X: (np array of len Nclips) input feature vectors
        t: (np array of len Nclips) targets; one hot vectors
        testFraction: (float) Nclips_test = testFraction * Nclips
    Outputs:
        X_train: training set
        X_test: test set
        t_train: training labels
        t_test: test labels
    """


    ### WRITE YOUR CODE HERE - 5 MARKS
    index=np.array(range(len(X)))
    No_train=int(len(X)*(1-testFraction))
    X_train=[]
    X_test=[]
    t_train=[]
    t_test=[]
    i=0
    for i in range(No_train):
        #np.random.seed(i)
        rand_n = np.random.choice(a = index)
        X_train.append(X[rand_n])
        t_train.append(t[rand_n])
        i= np.argwhere(index==rand_n)
        index = np.delete(index,i)
        
    for i in index:
        X_test.append(X[i])
        t_test.append(t[i])
        
    
    X_train=np.array(X_train)
    X_test=np.array(X_test)
    t_train=np.array(t_train)
    t_test=np.array(t_test)
        
        
    
    
    
    return X_train, t_train, X_test, t_test

In [8]:
def audio2mfcc(x, n_mfcc = 20, Fs = 16000):
    
    '''
    Compute Mel-frequency cepstral coefficients (MFCCs)
    Inputs:
        x: np array of shape (Nclips,)
        Fs: (int) sampling rate
        n_mfcc: (int) number of MFCC features
    Output:
        X: (np array) MFCC sequence
    '''

    ### WRITE YOUR CODE HERE - 3 MARKS
    X=[]
    for i in range(len(x)):
        a=lb.feature.mfcc(x[i],n_mfcc=n_mfcc,sr=Fs)
        X.append(a)
    X=np.array(X)

    return X 

In [9]:
class Classifier: 
    '''
    Create a linear classifier to classify each frame
    '''
    def __init__(self):
        self.W=np.zeros([20,1]) # define model parameters here
    
        
    def train(self,x_train, y_train):
        '''
        Train the linear classifier
        Inputs:
            x_train: training set
            y_train: training labels
        Output:
            None
        '''

        ### WRITE YOUR CODE HERE - 0 MARKS
        i=1
        X=x_train[0]
        music=np.array([1,0])
        speech=np.array([0,1])
        comparison=y_train[0]==music
        if comparison.all() :
            Y=np.ones([x_train[0].shape[1],1],dtype=int)
        else:
             Y=np.ones([x_train[0].shape[1],1],dtype=int)*(-1)
        
        for i in range(len(x_train)):
            X=np.concatenate((X,x_train[i]),axis=1)
            comparison=y_train[i]==music
            if comparison.all():
                a=np.ones([x_train[i].shape[1],1],dtype=int)
                Y=np.concatenate((Y,a),axis=0)
            else:
                s=np.ones([x_train[i].shape[1],1],dtype=int)*(-1)
                Y=np.concatenate((Y,s),axis=0,)
            
        X=np.transpose(X)
        T=np.transpose(X)
        
        self.W=np.dot(np.dot(np.linalg.inv(np.dot(T,X)),T),Y)
        

        
        return 
    
    def save_model(self, save_path):
        '''
        Save the trained model on local disk
        Input:
            save_path: location at which model is to be saved
        Output:
            None
        '''
        
        ### WRITE YOUR CODE HERE - 0 MARKS
        pickle.dump(Classifier,open('model_save','wb'))
            
        return
    
    def load_model(self, load_path):
        '''
        Save the trained model on local disk
        Input:
            load_path: location from which model is to be loaded
        Output:
            None
        '''
        
        ### WRITE YOUR CODE HERE - 0 MARKS
        loaded_model = pickle.load(open('model_save','rb'))
        
            
        return


    
    def predict_framewise(self,x_test):
        '''
        Framewise classification (speech or music)
        Input:
            x_test: test set
        Output:
            y_pred_framewise = framewise prediction
        '''
        

        ### WRITE YOUR CODE HERE - 5 MARKS
        y_pred_framewise=np.zeros([len(x_test) ,2,x_test.shape[2]],dtype=int)
        music=np.array([1,0])
        speech=np.array([0,1])
        for t in range(len(x_test)):
            for i in range(x_test[t].shape[1]):
                p=np.transpose(x_test[0][:,i])
                e=np.dot(p,self.W)
            if e>=0:
                y_pred_framewise[t][:,i]=np.transpose(music)
            if e<0:
                y_pred_framewise[t][:,i]=np.transpose(speech)
                
                
            
            


        return y_pred_framewise 
    
    def predict_aggregate(self,y_pred_framewise):
        '''
        Aggregate frames to give a single class label (music or speech) to the entire audio file
        Input:
            y_pred_framewise = framewise prediction
        Output:
            y_hat = frame aggregate (one-hot vectors)
        '''

        ### WRITE YOUR CODE HERE - 5 MARKS
        y_hat=np.zeros([len(y_pred_framewise),2],dtype=int)
        count_1up=0
        count_1down=0
        music=np.array([1,0])
        speech=np.array([0,1])
        for i in range(len(y_pred_framewise)):
            for c in range(y_pred_framewise[i].shape[1]):
                if y_pred_framewise[i][0][c]==1:
                    count_1up=count_1up+1
                else:
                    count_1down=count_1down+1
            if count_1up>count_1down:
                y_hat[i,:]=music
            else:
                y_hat[i,:]=speech
                
                
                    
            
            
            
        


        return y_hat

In [10]:
def computeCM(y, y_hat):
    '''
    Compute confusion matrix to evaluate your model
    Inputs:
        y = labels 
        y_hat = predicted output
    Output:
        confusion matrix: confusion matrix
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS
    K = len(np.unique(y))  
    confusion_matrix  = np.zeros([K, K],dtype=int)

    for i in range(len(y_hat)):
        if np.array_equal(y[i],y_hat[i]) and np.array_equal(y_hat[i],np.array([1,0])):
            confusion_matrix [0][0] += 1
        if np.array_equal(y[i],y_hat[i])!=True and np.array_equal(y_hat[i],np.array([1,0])):
            confusion_matrix [0][1] += 1
            
        if np.array_equal(y[i],y_hat[i]) and np.array_equal(y_hat[i],np.array([0,1])):
            confusion_matrix [1][1] += 1
        if np.array_equal(y[i],y_hat[i])!=True and np.array_equal(y_hat[i],np.array([0,1])):
            confusion_matrix [1][0] += 1
            



    return confusion_matrix 

In [14]:
if __name__=="__main__":
    Fs=16000
    # Read audio
    x_music = readDir('music_wavs')    #change it as per your directory
    x_speech = readDir('speech_wavs')  #change it as per your directory
    X = np.concatenate((x_music, x_speech))
    
    # Create labels
    y_music = np.array([[1,0]]*len(x_music))
    y_speech = np.array([[0,1]]*len(x_speech))
    Y = np.concatenate((y_music, y_speech))
    
    
    X_train, y_train, X_test, y_test = splitData(X, Y)
    
    # TRAINING 
    x_train = audio2mfcc(X_train)    # x_train: (Nclips, N_mfcc, N_frames)
    model = Classifier() 
    model.train(x_train, y_train)        # y_train: (Nclips, 2) -repeat it N_frames times inside the train
    
    # TESTING 
    x_test = audio2mfcc(X_test) 
    y_pred = model.predict_framewise(x_test)   # y_predict: (Nclips, 2, N_frames)
    y_hat = model.predict_aggregate(y_pred)    # y_hat: (Nclips, 2)
    
   # EVALUATION METRICS 
    confusion_matrix = computeCM(y_test, y_hat) 
    print(confusion_matrix) 

[[0 0]
 [3 1]]
